### 반복을 위한 Sacred
 - 목적  
    - 머신러닝 모델링에서 사용되는 config, parameter등을 손쉽게 저장할 수 있도록 도와주는 도구  

 - 이유  
    - 다양한 실험을 빠르게 진행하며, 손으로 기록하지 않고 자동으로 기록될 수 있도록 도와주는 도구  
    
 - 특징  
    - 환경설정을 다시 reproduce할 수 있도록 도와주는 도구
        - 실험의 모든 파라미터 추적
        - 여러 설정에 대해 쉽게 실험을 할 수 있음
        - DB에 각 실행의 설정을 저장
        - 결과를 reproduce  
        
    - 옴니보드등을 통해 저장된 값을 시각화할 수 있음
        - 옴니보드 : sacred에서 저장된 파리미터를 테이블로 보여주고 성능을 표시

#### sacred의 main mechanisms
- ConfigScopes : 함수의 local변수를 편리하게 다룰 수 있음. @ex.config 데코레이터로 사용
- Config Injection : 모든 함수에 있는 설정을 접근할 수 있음
- Command-line interface : 커맨드 라인으로 파라미터를 바꿔서 실행할 수 있음
- Observers : 실험의 모든 정보를 Observers에게 제공해 저장
- Automatic seeding : 실험의 무작위를 컨트롤할 때 도와줌

### 공식 홈페이지 sacred 예시

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from ipywidgets import interact
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import os
from numpy.random import permutation
from sklearn import svm, datasets
from sacred import Experiment
from sacred.observers import FileStorageObserver
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")
plt.rc("font",family="Malgun Gothic")
plt.rc("axes",unicode_minus=False)

plt.style.use('ggplot')
PROJECT_ID='new-york-taxi-309501' 

In [2]:
ex = Experiment('iris_rbf_svm', interactive=True)

@ex.config
def cfg():
    C = 1.0
    gamma = 0.7

@ex.main
def run(C, gamma):
    iris = datasets.load_iris()
    per = permutation(iris.target.size)
    iris.data = iris.data[per]
    iris.target = iris.target[per]
    clf = svm.SVC(C, 'rbf', gamma=gamma)
    clf.fit(iris.data[:90],
          iris.target[:90])
    return clf.score(iris.data[90:],
                   iris.target[90:])

- Notebook에선 interactive=True 지정해줘야 하고, 스크립트 형식이라면 이 옵션이 필요 없음  
- ex.main을 지정 => 이 때 cfg에 있는 인자들이 자동으로 injection됨  
- Notebook에선 ex.main을 쓰고 스크립트 파일에선 ex.automain을 사용

In [5]:
run_result = ex.run()

KeyError: 'brand'